In [1]:
# 1. Load environment variables and create the SQLAlchemy engine
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
from datetime import date, timedelta
import requests, json
from sqlalchemy import text

In [2]:
# Load .env from working directory
load_dotenv('.env')

True

In [3]:
# PostGIS connection vars
POSTGIS_USER = os.getenv('POSTGRES_USER')
POSTGIS_PASS = os.getenv('POSTGRES_PASSWORD')
POSTGIS_HOST = os.getenv('POSTGRES_HOST')
POSTGIS_PORT = os.getenv('POSTGRES_PORT')
POSTGIS_DB   = os.getenv('POSTGRES_DB')

# Create the engine
engine = create_engine(
    f"postgresql+psycopg2://"
    f"{POSTGIS_USER}:{POSTGIS_PASS}"
    f"@{POSTGIS_HOST}:{POSTGIS_PORT}/{POSTGIS_DB}"
)

In [4]:
url = "http://localhost:11434/api/chat"

In [5]:
# 1) Define the last 3 full days (excluding today)
end_exclusive = date.today()
start_date   = end_exclusive - timedelta(days=3)   # 3 days before today
date_list    = [start_date + timedelta(days=i) for i in range(3)]  
today = date.today()

# check
print(date_list)

[datetime.date(2025, 7, 9), datetime.date(2025, 7, 10), datetime.date(2025, 7, 11)]


In [6]:
prompt_template = """
Here are the SF 311 metrics for {date}:

**Request Type Breakdown**  
{request_table}

**Average resolution time:** {avg_time}  
**Open-case aging:** {aging}

Please write a concise, dashboard-style report that follows this structure exactly:

1. **Summary:** “{open_count} open vs. {closed_count} closed cases.”  
2. **Top 3 Request Types:** A bulleted list of the three request types.
3. **Closing:** One sentence commenting on the resolution speed ({avg_time}) and the open-case aging distribution ({aging}).

Use only the numbers shown above; do not add any extra interpretation. Keep it to 5–7 lines of plain text.
""".strip()

In [ ]:
md_lines = [
    f"# SF Castro 311 Dashboard Summaries ({date_list[0].isoformat()} → {date_list[-1].isoformat()})",
    ""
]

for dt in date_list:
    dt_str = dt.isoformat()

    # a) Pull status counts to get open/closed
    df_status = pd.read_sql(f"""
        SELECT status, COUNT(*) AS count
          FROM castro_311
         WHERE opened_ts::date = '{dt_str}'
         GROUP BY status;
    """, engine)

    open_count   = int(df_status.loc[df_status.status == 'Open',   'count'].sum() or 0)
    closed_count = int(df_status.loc[df_status.status == 'Closed', 'count'].sum() or 0)
    total = open_count + closed_count

    # b) Pull request_type counts
    df_req = pd.read_sql(f"""
        SELECT request_type, COUNT(*) AS count
          FROM castro_311
         WHERE opened_ts::date = '{dt_str}'
         GROUP BY request_type
         ORDER BY count DESC;
    """, engine)

    # c) Average resolution time
    avg_hours = pd.read_sql(f"""
        SELECT AVG(EXTRACT(EPOCH FROM (closed_ts - opened_ts)))/3600.0 AS avg_hours
          FROM castro_311
         WHERE opened_ts::date = '{dt_str}'
           AND closed_ts IS NOT NULL;
    """, engine).iloc[0,0] or 0
    avg_str = f"{avg_hours:.1f} hrs"

    # d) Open-case aging buckets
    a0, a1, a2 = pd.read_sql(f"""
        SELECT
          COUNT(*) FILTER (WHERE now() - opened_ts <= INTERVAL '1 day')          AS bucket_0_1,
          COUNT(*) FILTER (WHERE now() - opened_ts >  INTERVAL '1 day'
                         AND now() - opened_ts <= INTERVAL '3 days')            AS bucket_1_3,
          COUNT(*) FILTER (WHERE now() - opened_ts >  INTERVAL '3 days')         AS bucket_3_plus
        FROM castro_311
       WHERE opened_ts::date = '{dt_str}'
         AND closed_ts IS NULL;
    """, engine).iloc[0]
    aging_str = f"0–1 d={a0}, 1–3 d={a1}, 3+ d={a2}"

    # e) Raw metrics block
    req_table = df_req.to_markdown(index=False) if not df_req.empty else "*no request types*"
    md_lines += [
        f"## {dt_str}",
        "",
        "### Raw Metrics",
        "",
        f"• Total cases: {total}  ",
        f"• Open: {open_count}  ",
        f"• Closed: {closed_count}  ",
        "",
        "**Request Type Breakdown:**",
        "",
        req_table,
        "",
        f"**Average resolution time:** {avg_str}  ",
        f"**Open-case aging:** {aging_str}",
        ""
    ]

    # f) AI report
    if total == 0:
        ai_report = f"No updates or edits on {dt_str}."
    else:
        prompt = prompt_template.format(
            date=dt_str,
            total=total,
            open_count=open_count,
            closed_count=closed_count,
            request_table=req_table,
            avg_time=avg_str,
            aging=aging_str
        )
        resp = requests.post(url, json={
            "model":    "smollm2:1.7b",
            "stream":   False,
            "messages": [{"role": "user", "content": prompt}]
        })
        resp.raise_for_status()
        data = resp.json()

        if "choices" in data and data["choices"]:
            ai_report = data["choices"][0]["message"]["content"].strip()
        elif "message" in data:
            ai_report = data["message"]["content"].strip()
        else:
            ai_report = "_⚠️ AI returned empty response_"

    md_lines += [
        "### AI Report",
        "",
        ai_report,
        ""
    ]

# 4) Write out
output_path = "castro_ai_summary.md"
with open(output_path, "w") as f:
    f.write("\n".join(md_lines))

print(f"✅ Written {len(md_lines)} lines to '{output_path}' ({os.path.getsize(output_path)} bytes)")

In [ ]:
%%bash
# sync with remote (ignore pull errors)
git pull --rebase origin main 2>/dev/null || true

# stage all changes
git add -A

# commit and push only if there’s something new
if ! git diff --cached --quiet; then
  git commit -m "chore: update Castro AI summary"
  git push origin main
else
  echo "No new changes, skipping commit"
fi